# MCP Tools Example with sik-llms

This notebook demonstrates how to use the `sik-llms` library to connect to our MCP servers and interact with tools programmatically.

## Prerequisites

Before running this notebook, ensure your `.env` file has:
- `VITE_DEV_MODE=true` (bypasses auth)
- `OPENAI_API_KEY=sk-...` (for ReasoningAgent examples)

1. **Start Docker containers** (PostgreSQL + Redis):
   ```bash
   make docker-up
   ```

2. **Run database migrations** (if needed):
   ```bash
   make migrate
   ```

3. **Start the API server** (port 8000):
   ```bash
   make run
   ```

4. **Start the Content MCP server** (port 8001) - in a separate terminal:
   ```bash
   make content-mcp-server
   ```

5. **Start the Prompt MCP server** (port 8002) - in a separate terminal:
   ```bash
   make prompt-mcp-server
   ```

6. **Node.js** (for `npx` to run `mcp-remote`):
   - Ensure Node.js is installed
   - `npx` will automatically download `mcp-remote` on first use

## Configuration

In [1]:
from dotenv import load_dotenv

# Load .env from project root
load_dotenv()

# Dummy token - works with VITE_DEV_MODE=true (API doesn't validate)
PAT_TOKEN = "bm_devtoken"

# MCP Server URLs (local development)
CONTENT_MCP_URL = "http://localhost:8001/mcp"
PROMPT_MCP_URL = "http://localhost:8002/mcp"

## Connect to MCP Servers

The `MCPClientManager` from sik-llms uses the same config format as Claude Desktop. Since our MCP servers run over HTTP, we use `mcp-remote` as a stdio-to-HTTP bridge.

In [2]:
from sik_llms.mcp_manager import MCPClientManager

# Configuration using mcp-remote to bridge stdio to HTTP
mcp_config = {
    "mcpServers": {
        "content": {
            "command": "npx",
            "args": [
                "mcp-remote",
                CONTENT_MCP_URL,
                "--header",
                f"Authorization: Bearer {PAT_TOKEN}",
            ],
        },
        # "prompts": {
        #     "command": "npx",
        #     "args": [
        #         "mcp-remote",
        #         PROMPT_MCP_URL,
        #         "--header",
        #         f"Authorization: Bearer {PAT_TOKEN}",
        #     ],
        # },
    },
}

## List Available Tools

Connect to the MCP servers and discover available tools:

In [3]:
from IPython.display import Markdown # type: ignore

async with MCPClientManager(mcp_config, silent=False) as mcp:
    tools = mcp.get_tools()
    md = f"Found `{len(tools)}` tools\n\n"

    for tool in tools:
        md += f"### `{tool.name}`\n\n"
        if tool.description:
            md += f"{tool.description}\n\n"
        if tool.parameters:
            md += "**Arguments:**\n\n"
            for param in tool.parameters:
                required = "(required)" if param.required else "(optional)"
                md += f"- `{param.name}` {required}"
                if param.description:
                    md += f": {param.description}"
                md += "\n"
            md += "\n"
        md += "---\n\n"

    display(Markdown(md))

Found `8` tools

### `search_items`

Search across bookmarks and notes. By default searches both types. Use `type` parameter to filter to a specific content type. Returns metadata including content_length and content_preview (not full content).

**Arguments:**

- `query` (optional): Text to search in title, description, URL (bookmarks), and content
- `type` (optional): Filter by type: 'bookmark' or 'note'. Omit to search both.
- `tags` (optional): Filter by tags
- `tag_match` (optional): Tag matching: 'all' requires ALL tags, 'any' requires ANY tag
- `sort_by` (optional): Field to sort by
- `sort_order` (optional): Sort direction
- `limit` (optional): Maximum results to return
- `offset` (optional): Number of results to skip for pagination

---

### `get_item`

Get a bookmark or note by ID. By default includes full content. Use include_content=false to get content_length and content_preview for size assessment before loading large content.

**Arguments:**

- `id` (required): The item ID (UUID)
- `type` (required): Item type: 'bookmark' or 'note'
- `include_content` (optional): If true (default), include full content. If false, returns content_length and content_preview for size assessment.
- `start_line` (optional): Start line for partial read (1-indexed). Only valid when include_content=true.
- `end_line` (optional): End line for partial read (1-indexed, inclusive). Only valid when include_content=true.

---

### `edit_content`

Edit the 'content' field using string replacement. Use when: making targeted changes (small or large) where you can identify specific text to replace; adding, removing, or modifying a section while keeping the rest unchanged. More efficient than replacing entire content. Examples: fix a typo, add a paragraph, remove a section, update specific text. Does NOT edit title, description, or tags - only the main content body. The old_str must match exactly one location. Use search_in_content first to verify match uniqueness.

**Arguments:**

- `id` (required): The item ID (UUID)
- `type` (required): Item type: 'bookmark' or 'note'
- `old_str` (required): Exact text to find. Include surrounding context for uniqueness.
- `new_str` (required): Replacement text. Use empty string to delete the matched text.

---

### `search_in_content`

Search within a content item's text to find matches with line numbers and context. Use before editing to verify match uniqueness and build a unique old_str.

**Arguments:**

- `id` (required): The item ID (UUID)
- `type` (required): Item type: 'bookmark' or 'note'
- `query` (required): Text to search for (literal match)
- `fields` (optional): Fields to search (comma-separated): content, title, description. Default: 'content' only
- `case_sensitive` (optional): Case-sensitive search. Default: false
- `context_lines` (optional): Lines of context before/after each match (0-10). Default: 2

---

### `update_item`

Update metadata and/or fully replace content. Use when: updating metadata (title, description, tags, url); rewriting/restructuring where most content changes; changes are extensive enough that finding old_str is impractical. Safer for major rewrites. Examples: convert format (bullets to prose), change tone/audience, reorganize structure, complete rewrite, update tags. All parameters optional - only provide what you want to change (at least one required). NOTE: For targeted edits where you can identify specific text to replace, use edit_content instead - it's more efficient for surgical changes.

**Arguments:**

- `id` (required): The item ID (UUID)
- `type` (required): Item type: 'bookmark' or 'note'
- `description` (optional): New description. Omit to leave unchanged.
- `tags` (optional): New tags (replaces all existing tags). Omit to leave unchanged.
- `url` (optional): New URL (bookmarks only). Omit to leave unchanged.
- `content` (optional): New content (FULL REPLACEMENT of entire content field). Omit to leave unchanged.
- `expected_updated_at` (optional): For optimistic locking. If provided and the item was modified after this timestamp, returns a conflict error with the current server state. Use the updated_at from a previous response.

---

### `create_bookmark`

Create a new bookmark.

**Arguments:**

- `url` (required): The URL to bookmark
- `title` (optional): Bookmark title
- `description` (optional): Bookmark description
- `tags` (optional): Tags to assign (lowercase with hyphens, e.g., 'machine-learning')

---

### `create_note`

Create a new note.

**Arguments:**

- `title` (required): The note title (required)
- `description` (optional): Short description/summary
- `content` (optional): Markdown content of the note
- `tags` (optional): Tags to assign (lowercase with hyphens, e.g., 'meeting-notes')

---

### `list_tags`

List all tags with their usage counts

---



## Call Tools Directly

You can call tools directly using `call_tool()`. This returns a `CallToolResult` (MCP protocol) with:

- `content`: List of content items (e.g., `TextContent` with the JSON response as text)
- `structuredContent`: Optional dict with parsed/typed data (FastMCP populates this automatically)
- `isError`: Boolean indicating if the tool call failed

In [4]:
import json

async with MCPClientManager(mcp_config) as mcp:
    # Example: List all tags
    result = await mcp.call_tool("list_tags", {})

    print("=== CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== content (list of TextContent) ===\n")
    for i, content_item in enumerate(result.content):
        print(f"[{i}] type: {content_item.type}")
        print(f"[{i}] text: {content_item.text}\n")

    print("=== structuredContent (parsed dict) ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== CallToolResult ===

isError: False

=== content (list of TextContent) ===

[0] type: text
[0] text: {"tags":[{"name":"eval-test","count":3},{"name":"documents","count":1},{"name":"mcp-eval","count":1},{"name":"my-list-tag","count":1},{"name":"test","count":1},{"name":"writing","count":1}]}

=== structuredContent (parsed dict) ===

{
  "tags": [
    {
      "name": "eval-test",
      "count": 3
    },
    {
      "name": "documents",
      "count": 1
    },
    {
      "name": "mcp-eval",
      "count": 1
    },
    {
      "name": "my-list-tag",
      "count": 1
    },
    {
      "name": "test",
      "count": 1
    },
    {
      "name": "writing",
      "count": 1
    }
  ]
}


In [5]:
async with MCPClientManager(mcp_config) as mcp:
    # Example: Search for bookmarks
    result = await mcp.call_tool("search_items", {
        "limit": 2,
    })

    print("=== CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== content (list of TextContent) ===\n")
    for i, content_item in enumerate(result.content):
        print(f"[{i}] type: {content_item.type}")
        print(f"[{i}] text: {content_item.text[:200]}..." if len(content_item.text) > 200 else f"[{i}] text: {content_item.text}")  # noqa: E501
        print()

    print("=== structuredContent (parsed dict) ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== CallToolResult ===

isError: False

=== content (list of TextContent) ===

[0] type: text
[0] text: {"items":[{"type":"note","id":"019bf721-8237-7889-83f3-eb31d538d5c7","title":"MCP Eval Test Note","description":null,"tags":["test","mcp-eval"],"created_at":"2026-01-25T21:48:47.032091Z","updated_at":...

=== structuredContent (parsed dict) ===

{
  "items": [
    {
      "type": "note",
      "id": "019bf721-8237-7889-83f3-eb31d538d5c7",
      "title": "MCP Eval Test Note",
      "description": null,
      "tags": [
        "test",
        "mcp-eval"
      ],
      "created_at": "2026-01-25T21:48:47.032091Z",
      "updated_at": "2026-01-25T21:48:47.032091Z",
      "last_used_at": "2026-01-25T21:48:47.032091Z",
      "deleted_at": null,
      "archived_at": null,
      "content_length": 106,
      "content_preview": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",
      "url": null,
      "version": 1,
      "name": null

## Create Test Data

Let's create a note that we can use to test editing:

In [6]:
async with MCPClientManager(mcp_config) as mcp:
    # Create a test note with a deliberate typo
    result = await mcp.call_tool("create_note", {
        "title": "MCP Eval Test Note",
        "content": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",  # noqa: E501
        "tags": ["test", "mcp-eval"],
    })
    if result.isError:
        print(f"Error: {result.content[0].text}")
    else:
        note_data = json.loads(result.content[0].text)
        note_id_original = note_data["id"]
        print(f"Created note with ID: {note_id_original}")
        print(f"Title: {note_data['title']}")
        print(f"Content: {note_data['content']}")

Created note with ID: 019bf721-c8c7-7f35-9f36-13bb58157aa8
Title: MCP Eval Test Note
Content: This is a tset note for evaluating MCP tools.

It has a typo that we will fix using the edit_content tool.


## Test Content Editing Tools

Now let's use the new editing tools to fix the typo:

In [8]:
# First, search for the note we created
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("search_items", {
        "query": "MCP Eval Test Note",
        "limit": 1,
    })

    print("=== search_items CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

    # Extract note_id for subsequent cells
    if result.structuredContent.get("items"):
        note_id = result.structuredContent["items"][0]["id"]
        print(f"\nExtracted note_id: {note_id}")
    else:
        print("\nNote not found - run the create cell first")
        note_id = None

assert note_id_original == note_id, "Note ID from search does not match created note ID"

=== search_items CallToolResult ===

isError: False

=== structuredContent ===

{
  "items": [
    {
      "type": "note",
      "id": "019bf721-c8c7-7f35-9f36-13bb58157aa8",
      "title": "MCP Eval Test Note",
      "description": null,
      "tags": [
        "test",
        "mcp-eval"
      ],
      "created_at": "2026-01-25T21:49:05.096705Z",
      "updated_at": "2026-01-25T21:49:05.096705Z",
      "last_used_at": "2026-01-25T21:49:05.096705Z",
      "deleted_at": null,
      "archived_at": null,
      "content_length": 106,
      "content_preview": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",
      "url": null,
      "version": 1,
      "name": null,
      "arguments": null
    }
  ],
  "total": 2,
  "offset": 0,
  "limit": 1,
  "has_more": true
}

Extracted note_id: 019bf721-c8c7-7f35-9f36-13bb58157aa8


In [10]:
# Get the note content
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("get_item", {
        "id": note_id,
        "type": "note",
    })

    print("=== get_item CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== get_item CallToolResult ===

isError: False

=== structuredContent ===

{
  "id": "019bf721-c8c7-7f35-9f36-13bb58157aa8",
  "title": "MCP Eval Test Note",
  "description": null,
  "tags": [
    "test",
    "mcp-eval"
  ],
  "created_at": "2026-01-25T21:49:05.096705Z",
  "updated_at": "2026-01-25T21:49:05.096705Z",
  "last_used_at": "2026-01-25T21:49:05.096705Z",
  "deleted_at": null,
  "archived_at": null,
  "content_length": 106,
  "content_preview": null,
  "content": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",
  "content_metadata": {
    "total_lines": 3,
    "start_line": 1,
    "end_line": 3,
    "is_partial": false
  }
}


In [11]:
# Search within the content to find the typo
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("search_in_content", {
        "id": note_id,
        "type": "note",
        "query": "tset",
    })

    print("=== search_in_content CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== search_in_content CallToolResult ===

isError: False

=== structuredContent ===

{
  "matches": [
    {
      "field": "content",
      "line": 1,
      "context": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool."
    }
  ],
  "total_matches": 1
}


In [12]:
# Fix the typo using edit_content
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("edit_content", {
        "id": note_id,
        "type": "note",
        "old_str": "tset note",
        "new_str": "test note",
    })

    print("=== edit_content CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== edit_content CallToolResult ===

isError: False

=== structuredContent ===

{
  "response_type": "minimal",
  "match_type": "exact",
  "line": 1,
  "data": {
    "id": "019bf721-c8c7-7f35-9f36-13bb58157aa8",
    "updated_at": "2026-01-25T21:49:41.726702Z"
  }
}


In [13]:
from sik_llms import (
    ReasoningAgent,
    user_message,
    ToolPredictionEvent,
    ToolResultEvent,
    ThinkingEvent,
    TextResponse,
)

async with MCPClientManager(mcp_config) as mcp:
    # Get tools from MCP servers
    tools = mcp.get_tools()

    # Create a reasoning agent with the MCP tools
    agent = ReasoningAgent(
        model_name="gpt-4.1-mini",
        tools=tools,
        max_iterations=10,
    )
    # Ask the agent to perform a task
    messages = [
        user_message("Find my note titled 'MCP Eval Test Note' and tell me what it's about."),
    ]

    print("Agent reasoning...\n")
    async for event in agent.stream(messages):
        if isinstance(event, ThinkingEvent):
            print(f"[Thinking] {event.content[:200]}..." if len(event.content) > 200 else f"[Thinking] {event.content}")  # noqa: E501
        elif isinstance(event, ToolPredictionEvent):
            print(f"[Tool Call] {event.name}({event.arguments})")
        elif isinstance(event, ToolResultEvent):
            result_preview = str(event.result)[:200]
            print(f"[Tool Result] {result_preview}..." if len(str(event.result)) > 200 else f"[Tool Result] {event.result}")  # noqa: E501
        elif isinstance(event, TextResponse):
            print(f"\n[Final Answer]\n{event.response}")
            print(f"\n[Tokens] Input: {event.input_tokens}, Output: {event.output_tokens}")

Agent reasoning...

[Thinking] 
[Thinking] I need to find the note titled 'MCP Eval Test Note' in the user's notes to check its content and determine what it is about.
[Tool Call] search_items({'query': 'MCP Eval Test Note', 'type': 'note', 'limit': 1})
[Tool Result] {"items":[{"id":"019bf721-c8c7-7f35-9f36-13bb58157aa8","title":"MCP Eval Test Note","description":null,"tags":["test","mcp-eval"],"created_at":"2026-01-25T21:49:05.096705Z","updated_at":"2026-01-25T21...
[Thinking] I found the note titled 'MCP Eval Test Note'. The content preview shows the note is a test note for evaluating MCP tools and mentions it has a typo to be fixed using the edit_content tool. Based on th...

[Final Answer]
The note titled "MCP Eval Test Note" is a test note created for evaluating MCP tools. It includes a mention that there is a typo in the note which will be fixed using an editing tool. Essentially, the note serves as a test to assess the capabilities of the MCP system, particularly focusing on con

## Example: Agent Updating Note Content

In [14]:
async with MCPClientManager(mcp_config) as mcp:
    tools = mcp.get_tools()

    agent = ReasoningAgent(
        model_name="gpt-4.1-mini",
        tools=tools,
        max_iterations=15,  # More iterations for multiple edits
    )

    messages = [
        user_message(
            "Find the note titled 'MCP Eval Test Note' replace any mention of tool/tools with bold formatting.",  # noqa: E501
        ),
    ]

    print("Agent working on fixing typos...\n")
    async for event in agent.stream(messages):
        if isinstance(event, ThinkingEvent):
            print(f"[Thinking] {event.content}")
        elif isinstance(event, ToolPredictionEvent):
            print(f"[Tool Call] {event.name}")
            print(f"  Arguments: {json.dumps(event.arguments, indent=4)}")
        elif isinstance(event, ToolResultEvent):
            print(f"[Tool Result] {event.name}:")
            try:
                result_data = json.loads(event.result)
                print(f"  {json.dumps(result_data, indent=4)}")
            except (json.JSONDecodeError, TypeError):
                print(f"  {event.result}")
        elif isinstance(event, TextResponse):
            print(f"\n[Final Answer]\n{event.response}")
            print(f"\n[Tokens] Input: {event.input_tokens}, Output: {event.output_tokens}")

Agent working on fixing typos...

[Thinking] 
[Thinking] I need to find the note titled 'MCP Eval Test Note' so I can locate the mentions of the word 'tool' or 'tools' and replace them with the same words in bold formatting.
[Tool Call] search_items
  Arguments: {
    "query": "MCP Eval Test Note",
    "type": "note",
    "limit": 1
}
[Tool Result] search_items:
  {
    "items": [
        {
            "id": "019bf721-c8c7-7f35-9f36-13bb58157aa8",
            "title": "MCP Eval Test Note",
            "description": null,
            "tags": [
                "test",
                "mcp-eval"
            ],
            "created_at": "2026-01-25T21:49:05.096705Z",
            "updated_at": "2026-01-25T21:49:41.726702Z",
            "last_used_at": "2026-01-25T21:49:05.096705Z",
            "deleted_at": null,
            "archived_at": null,
            "content_length": 106,
            "content_preview": "This is a test note for evaluating MCP tools.\n\nIt has a typo that we will fix

## Cleanup

Delete the test notes via the API:

```bash
# Permanently delete a specific note
curl -X DELETE "http://localhost:8000/notes/{note_id}?permanent=true" \
  -H "Authorization: Bearer bm_devtoken"
```

Or filter by tag `test` or `mcp-eval` in the web UI to find and delete test notes.

In [15]:
import httpx

async with httpx.AsyncClient() as client:
    response = await client.delete(
        f"http://localhost:8000/notes/{note_id}?permanent=true",
        headers={"Authorization": f"Bearer {PAT_TOKEN}"},
    )
    if response.status_code == 204:
        print(f"Deleted note {note_id}")
    else:
        print(f"Failed to delete: {response.status_code} - {response.text}")

Deleted note 019bf721-c8c7-7f35-9f36-13bb58157aa8


---